In [ ]:
import torch
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

In [ ]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
print(src_path)
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2NativeDataType, FFF2NativeDataType, SingleTIFF2NativeDataType

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion, IlluminationCorrection, \
                                Calculate_BIGFISH_Threshold, CellProperties, Automatic_BIGFISH_Threshold, GeneralCellposeSegmentation

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5, \
                                     remove_local_data

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

In [3]:
scope = ScopeClass() 
data = DataContainer(clear_after_error=False)
settings = Settings(name='demo')
experiment = Experiment()

In [ ]:
# you can change the parameters
experiment.initial_data_location = r'Users/Jack/HistoneModification_ScratchAssay/Full 24 hour scratch assay.tif'

experiment.nucChannel = 0
experiment.cytoChannel = 0
experiment.voxel_size_z = 500

settings.num_chunks_to_run = 5 # this is the limit to how many chunks you want to run and it will stop after that
settings.sigma_dict = {0: 50, 1: 100, 2: 100}
settings.mask_structure = {'cellMasks': ('ptczyx', None, None),
                          'nucMasks': ('ptczyx', None, None),
                            'cell_mask': ('zyx', 'cytoChannel', 'cellMasks'), 
                            'nuc_mask': ('zyx', 'nucChannel', 'nucMasks')}

Parameters().get_parameters()

In [ ]:
SingleTIFF2NativeDataType().run()

In [ ]:
Display().displayImage_maxProject(channel=0)

In [ ]:
settings.cellpose_min_size = 500
settings.cellpose_diameter = [180, 90]
settings.cellpose_pretrained_model = ["histone_cyto", 'histone_nuclei'] 

SimpleCellposeSegmentaion().run()

In [ ]:
CellProperties().run()